In [1]:
"""
    Maps Standings and Fixtures for building the Training Data
"""

'\n    Maps Standings and Fixtures for building the Training Data\n'

In [2]:
import json
import os
import pandas as pd
from pandas import json_normalize

In [17]:
# This is just a test
df = pd.DataFrame()
fixture = 'fixture.json'
team_home = dirName + '720/720.json'
team_away = dirName + '529/529.json'

with open(fixture) as f:
    my_fixture = json.load(f)
with open(team_home) as f:
    standings_home = json.load(f)
    del standings_home['stats_away']
    my_fixture.update({"team_home": standings_home})
with open(team_away) as f:
    standings_away = json.load(f)
    del standings_home['stats_home']
    my_fixture.update({"team_away": standings_away})
df = df.append(json_normalize(my_fixture))
df.head()

,goals_home,goals_away,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,team_home.goals_diff,team_home.form,team_home.description,team_away.team.id,...,stats_away.fouls,stats_away.corners,stats_away.offside,stats_away.possession,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage
0,0,1,720,Valladolid,14,39,-11,LLWDD,None,529,...,13,5,None,65%,2,None,4,732,643,88%


In [16]:
for col in df.columns: 
    print(col)

goals_home
goals_away
team_home.team.id
team_home.team.name
team_home.rank
team_home.points
team_home.goals_diff
team_home.form
team_home.description
team_away.team.id
team_away.team.name
team_away.rank
team_away.points
team_away.goals_diff
team_away.form
team_away.description
team_away.stats_home.played
team_away.stats_home.wins
team_away.stats_home.draws
team_away.stats_home.lose
team_away.stats_home.goals_for
team_away.stats_home.avg_goals_for
team_away.stats_home.goals_against
team_away.stats_home.avg_goals_against
team_away.stats_home.clean_sheet
team_away.stats_home.failed_to_score
team_away.stats_home.streaks.wins
team_away.stats_home.streaks.draws
team_away.stats_home.streaks.loses
team_away.stats_home.streaks.best_win
team_away.stats_home.streaks.best_lose
team_away.stats_home.streaks.best_goals_for
team_away.stats_home.streaks.best_goals_against
team_away.stats_away.played
team_away.stats_away.wins
team_away.stats_away.draws
team_away.stats_away.lose
team_away.stats_away.goal

In [4]:
league = '140'
season = '2019'

In [5]:
# League directory
dirName = league + '/' + season + '/' 

In [3]:
# Multi League directory
country = 'GB'
season = '2019'
dirCountry = '../' + country + '/'

In [20]:
df = pd.DataFrame()
standings = None
away_team_stands = None
fixture = None
# This will prevent duplicated fixtures
visited_fixtures = []
# This will help to map to standings
visited_standings = {}

# Principal leagues are sorted ascendent
dirLeague = sorted(os.listdir(dirCountry))

for league in dirLeague:
    if league == ".DS_Store":
        continue
    if league == "teams.json":
        continue
    dirName = dirCountry + league + '/' + season + '/' 
    for subdir in os.listdir(dirName):
        if subdir == ".DS_Store":
            continue
        team_id = subdir
        # Entering directory of a team
        dirTeam = dirName + team_id + '/'

        # Gets standings from a team if they have not being added
        if team_id not in visited_standings:
            json_standings = dirTeam + team_id + '.json'
            with open(json_standings) as f:
                standings = json.load(f)
                visited_standings[team_id] = standings
        else:
            standings = visited_standings[team_id]

        # Directory of team in home
        dirHome = dirTeam + 'home/'
        for file in os.listdir(dirHome):
            # Check if fixture hasn't being added to dataframe
            if file not in visited_fixtures:
                path = dirHome + file
                with open(path) as f:
                    fixture = json.load(f)
                    # Adds id of fixture as "id_fixture.json"
                    visited_fixtures.append(file)
                    # Next, get away team's external info
                    away_team = str(fixture["team_away"]["id"])
                    if away_team in visited_standings:
                        away_team_stands = visited_standings[away_team]
                    else:
                        # Get standings from away_team
                        fileAwayTeam = dirName + away_team + '/' + away_team + '.json'
                        with open(fileAwayTeam) as f:
                            away_team_stands = json.load(f)
                            visited_standings[away_team] = away_team_stands
                # Three main jsons to a dataframe
                # standings "this is the home team"
                # away_team_stands
                # the fixture itself
                # The premise that away standings from home team aren't relevant
                try:
                    #del standings['stats_away']
                    fixture.update({"team_home": standings})
                except:
                    print("Couldn't delete stats away")
                # The premise that home standings from away team aren't relevant
                try:
                    #del away_team_stands['stats_home']
                    fixture.update({"team_away": away_team_stands})
                except:
                    print("Couldn't delete stats home")
                # Adding league
                fixture.update({"league": int(league)})
                df = df.append(json_normalize(fixture))
            else:
                continue

        # Directory of team in away
        dirAway = dirTeam + 'away/'
        for file in os.listdir(dirHome):
            # Check if fixture hasn't being added to dataframe
            if file not in visited_fixtures:
                path = dirAway + file
                with open(dirAway) as f:
                    fixture = json.load(f)
                    # Adds id of fixture as "id_fixture.json"
                    visited_fixtures.append(file)
                    # Next, get home team's external info
                    home_team = str(fixture["team_home"]["id"])
                    if home_team in visited_standings:
                        home_team_stands = visited_standings[home_team]
                    else:
                        # Get standings from home_team
                        fileHomeTeam = dirName + home_team + '/' + home_team + '.json'
                        with open(fileHomeTeam) as f:
                            home_team_stands = json.load(f)
                            visited_standings[home_team] = home_team_stands
                # Three main jsons to a dataframe
                # standings "this is the away team"
                # home_team_stands
                # the fixture itself
                # The premise that away standings from home team aren't relevant
                try:
                    #del home_team_stands['stats_away']
                    fixture.update({"team_home": home_team_stands})
                except:
                    print("Couldn't delete stats away")
                # The premise that home standings from away team aren't relevant
                try:
                    #del standings['stats_home']
                    fixture.update({"team_away": standings})
                except:
                    print("Couldn't delete stats home")
                # Adding league
                fixture.update({"league": int(league)})
                df = df.append(json_normalize(fixture))
            else:
                continue

In [21]:
df.head()

,goals_home,goals_away,league,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,team_home.goals_diff,team_home.form,team_home.description,...,stats_away.offside,stats_away.possession,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage,stats_home,stats_away
0,1,3,39,66,Aston Villa,19,31,-27,DWLLL,Relegation - Championship,...,3,44%,1,None,5,341.0,234.0,69%,NaN,NaN
0,2,0,39,66,Aston Villa,19,31,-27,DWLLL,Relegation - Championship,...,1,65%,3,None,1,539.0,435.0,81%,NaN,NaN
0,2,2,39,66,Aston Villa,19,31,-27,DWLLL,Relegation - Championship,...,None,40%,4,None,1,281.0,177.0,63%,NaN,NaN
0,0,0,39,66,Aston Villa,19,31,-27,DWLLL,Relegation - Championship,...,3,54%,3,1,5,470.0,391.0,83%,NaN,NaN
0,1,2,39,66,Aston Villa,19,31,-27,DWLLL,Relegation - Championship,...,2,38%,2,None,6,295.0,208.0,71%,NaN,NaN


In [23]:
df.shape

(395, 119)

In [27]:
# csv to save
file_title = 'premier.csv'
csv_file = dirCountry + file_title

In [28]:
df.to_csv(csv_file)

In [51]:
os.listdir(dirName)

['533',
 '798',
 '532',
 '.DS_Store',
 '543',
 '715',
 '529',
 '545',
 '542',
 '537',
 '530',
 '539',
 '538',
 '531',
 '536',
 '540',
 '541',
 '546',
 '548',
 '720',
 '727']